# Finding Lane Lines

## Import Packages

In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from statistics import median
import cv2
import math
%matplotlib inline

## Helper Functions

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def fit_best_line(X,Y):
    """
    Returns the best line that can fit a set of points
    """
    [m,b] =np.polyfit(X, Y, 1)
    return m,b

def extrapolate_X(y,m,b):
    """
    Returns extrapolated value of x coordinate given the y coordinate for a line
    """
    x = int((y - b)/m)
    return x

def calc_extrapolated_line(lines_list,y_start,y_end):
    """
    Fit the best line for a set of points and extrapolate the start and end of the line
    """
    X = []
    Y = []
    for line in lines_list:
        for x1,y1,x2,y2 in line:
            X.append(x1)
            X.append(x2)
            Y.append(y1)
            Y.append(y2)

    [m,b] =fit_best_line(X,Y)
        
    x = extrapolate_X(y_start,m,b)
    start_point = (x,y_start)

    x = extrapolate_X(y_end,m,b)
    end_point = (x,y_end)
    

    return start_point,end_point
    
def calc_merged_point(point1,point2,weight):
    """
    Apply complementry filter for two points given the weight
    return the merged point
    """
    merged_x = int((point1[0] * weight) + (point2[0] * (1-weight)))
    merged_y = int((point1[1] * weight) + (point2[1] * (1-weight)))
    return (merged_x,merged_y)

def draw_lines(img, lines, color=[255, 0, 0], thickness=7):
    """    
    This fuction Filter the hough transform lines and apply a simple complementry filter
    to merge the new data with the old data
    
    Then the function draws the merged lines with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    """    
    meanSlopeRight = 0.5 #Mean value for the right lane slope
    meanSlopeLeft = -0.6 #Mean value for the left lane slope
    stdDev = 0.1 #slope standard deviation
    filtered_right_lines = []
    filtered_left_lines = []
    
    #Filtering step based on slope
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = math.atan2(y2-y1,x2-x1)
            if(slope > 0.0 and slope < meanSlopeRight + stdDev and slope > meanSlopeRight - stdDev):
                filtered_right_lines.append(line)
                
            if(slope < 0.0 and slope < meanSlopeLeft + stdDev and slope > meanSlopeLeft - stdDev):
                filtered_left_lines.append(line)
    
    #Apply best line fit algorithm to the filtered points
    #Apply Compelementry filter to merge the new lane lines with the old lane lines
    
    y_start = int(img.shape[0]/2 + 80)
    y_end = img.shape[0]
    
    #Right Lane Merge and Plotting
    global start_point_right,end_point_right
    if(len(filtered_right_lines) > 3):
        start_point,end_point = calc_extrapolated_line(filtered_right_lines,y_start,y_end)
        
        if(start_point_right[0] == 0.0 and start_point_right[1] == 0.0 and
           end_point_right[0] == 0.0 and end_point_right[1] == 0.0):
            
            start_point_right = start_point
            end_point_right = end_point
        else:
            start_point_right = calc_merged_point(start_point_right,start_point,0.7)
            end_point_right = calc_merged_point(end_point_right,end_point,0.7)
        
        cv2.line(img, start_point_right, end_point_right, color, thickness)
        
    else:
        cv2.line(img, start_point_right, end_point_right, color, thickness)
    
    #Left Lane Merge and Plotting
    global start_point_left,end_point_left
    if(len(filtered_left_lines) > 3):
        start_point,end_point = calc_extrapolated_line(filtered_left_lines,y_start,y_end)
        
        if(start_point_left[0] == 0.0 and start_point_left[1] == 0.0 and
           end_point_left[0] == 0.0 and end_point_left[1] == 0.0):
            
            start_point_left = start_point
            end_point_left = end_point
        else:
            start_point_left = calc_merged_point(start_point_left,start_point,0.7)
            end_point_left = calc_merged_point(end_point_left,end_point,0.7)
            
        cv2.line(img, start_point_left, end_point_left, color, thickness)
        
    else:
        cv2.line(img, start_point_left, end_point_left, color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Pipeline

In [3]:
def process_image(image):
    #Transform colored image to gray scale
    grayImage = grayscale(image)

    #Apply Gaussian Blur to smooth the image
    smoothImage = gaussian_blur(grayImage,3)
    
    #Apply Canny Edge Detector
    lowThreshold = 150
    highThreshold = 200
    cannyEdges = canny(smoothImage, lowThreshold, highThreshold)
    
    #Apply a four sided polygon mask
    x = image.shape[1]/2
    y = image.shape[0]/2
        
    upperLeft = (x -50 ,y +70)
    upperRight = (x + 50 ,y +70)
    lowerLeft = (50 ,image.shape[0])
    lowerRight = (image.shape[1]-50,image.shape[0])
    
    vertices = np.array([[lowerLeft,upperLeft, upperRight, lowerRight]], dtype=np.int32)
    maskedEdges = region_of_interest(cannyEdges,vertices)
    
    # Define the Hough transform parameters
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 30  # minimum number of votes (intersections in Hough grid cell)
    minLineLength = 20 #minimum number of pixels making up a line
    maxLineGap = 10 # maximum gap in pixels between connectable line segments

    # Run Hough on edge detected image
    # Iterate over the output "lines" and draw lines on a blank image
    linesImage = hough_lines(maskedEdges, rho, theta, threshold,minLineLength, maxLineGap)
    
    # Draw the lines on the edge image
    result = cv2.addWeighted(image, 0.8, linesImage, 1, 0) 
    
    return result

## Vedio Stream Input and Processing

In [4]:
#intialize global variables
start_point_right = (0,0)
end_point_right = (0,0)
start_point_left = (0,0)
end_point_left = (0,0)

white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|███████████████████████████████████████▊| 221/222 [00:04<00:00, 54.03it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 4.63 s


## Processed Vedio Result

In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Video Stream Input and Processing

In [6]:
#intialize global variables
start_point_right = (0,0)
end_point_right = (0,0)
start_point_left = (0,0)
end_point_left = (0,0)

yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
yellow_clip = clip1.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████▉| 681/682 [00:11<00:00, 60.96it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 11.6 s


## Processed Vedio Result

In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Vedio Stream Input and Processing

In [8]:
#intialize global variables
start_point_right = (0,0)
end_point_right = (0,0)
start_point_left = (0,0)
end_point_left = (0,0)

challenge_output = 'test_videos_output/challenge.mp4'
clip1 = VideoFileClip("test_videos/challenge.mp4")
challenge_clip = clip1.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|████████████████████████████████████████| 251/251 [00:07<00:00, 31.79it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 8.86 s


## Processed Vedio Result

In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))